# Assignment 3:  Filtered Vocabulary

* Janghyuk Boo 40008835 <br/>
* Jixuan Li    40073785

In [6]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import  word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import csv

data = pd.read_csv ("covid_training.tsv", sep = '\t')
extracted=data[['text','q1_label']]
# stopWords = nltk.corpus.stopwords.words('english')
def preprocess(text): 
    tokenized = nltk.word_tokenize(text.lower())
#     removedStopWords = [w for w in tokenized if w not in stopWords]
#     punctual_removed = [w for w in removedStopWords if w.isalpha()]
    punctual_removed = [w for w in tokenized if w.isalpha()]
    return punctual_removed

lower= extracted['text'].apply(lambda text: preprocess(text))
def convert_tuple_str(tup):
  return ' '.join(tup)
def convert_tuple_str2(tup):
    lists=list()
    for i in tup:
        a=' '.join(i)
        lists.append(a)
    return lists

strings= lower.apply(lambda row: convert_tuple_str(row) )
out = ' '.join(strings).split(' ')


from collections import defaultdict
d=defaultdict(int)
for lister in lower:
    for item in lister:
        d[item]+=1
        
#remove first occurance
tokens=[key for key,value in d.items() if value>1]
texts = [[word for word in document if word in tokens] for document in lower]
out=convert_tuple_str2(texts)
out= ' '.join(out).split(' ')

In [7]:
from nltk import FreqDist
f_dist1 =FreqDist(out)
vocab=dict(f_dist1)
len(vocab)

1091

In [8]:
smooth=0.01
yes = {}
no = {}
factual_count=0
not_factual_count=0

for term in vocab:
    yes[term] = smooth
    no[term] = smooth
    
condition_yes = {}
condition_no = {}

digit=extracted['q1_label'].map(dict(yes=1, no=0))
print(digit.value_counts())
num_yes=digit.value_counts()[1]
num_no=digit.value_counts()[0]
tweet_count=num_yes+num_no
def counting(text,target):
    if target ==1:
        dic= yes
    else:
        dic= no
    for token in text.split(' '):
        if token in dic:
            dic[token]+=1
    
with open("covid_training.tsv", encoding="utf8") as file:
    is_first_line = True
    reader = csv.reader(file, delimiter='\t')

    for row in reader:
        if is_first_line:
            is_first_line = False
            continue
        fact = 0
        if row[2].lower() == 'yes':
            fact = 1
        else:
            fact = 0
        
        tweet_text = row[1].lower()
        counting(tweet_text, fact)
    prob_is_factual = np.log10(num_yes / tweet_count)
    prob_is_not_factual = np.log10(num_no / tweet_count)

    for term in yes:
        condition_yes[term] = np.log10(yes[term] / (num_yes + (len(vocab) * smooth)) )#smoothed_yes
    for term in no:
        condition_no[term] = np.log10(no[term] / (num_no + (len(vocab) * smooth)))#smoothed_no

print("FV vocab size",len(vocab))

1    247
0    152
Name: q1_label, dtype: int64
FV vocab size 1091


In [9]:
test_result = {}
correct_counter = 0
with open("covid_test_public.tsv", encoding='utf8') as f:
    test_set = csv.reader(f, delimiter='\t')
    for line in test_set:
        test_tweet_id = line[0]
        test_tweet_text = preprocess(line[1])
        test_tweet_target = line[2]
        yes_score = prob_is_factual
        no_score = prob_is_not_factual
        for token in test_tweet_text:
            cur_token = token
            if cur_token in condition_yes:
                yes_score += condition_yes[cur_token]
                no_score += condition_no[cur_token]
            else:
                yes_score += np.log10(0.01/(num_yes + (len(vocab) * smooth)))
                no_score += np.log10(0.01/(num_no + (len(vocab) * smooth)))
        prediction = 'no'
        final_score = no_score
        if yes_score>= no_score:
            prediction = 'yes'
            final_score = yes_score
        correctness = 'wrong'
        if test_tweet_target == prediction:
            correctness = 'correct'
            correct_counter+=1
        innerString = test_tweet_id+ "  " + prediction + "  " + str(final_score) + "  "+test_tweet_target+"  "+ correctness
        test_result[test_tweet_id] = innerString

In [10]:
with open('./trace_NB-BOW-FV.txt', 'w') as f:
    for content in test_result.values():
        f.write(content)
        f.write('\n')
    f.close()

In [11]:
correct_count=0
incorrect_count=0
predicted_count=0
predicted_wrong_count=0
total_factual=0
total_not_factual=0

count=0

with open("trace_NB-BOW-FV.txt", encoding='utf8') as f:
    test_set = csv.reader(f, delimiter=' ')
    for line in test_set:
        prediction=line[2]
        answer=line[6]
        correct=line[8]
        if correct=='correct':
            if answer == "yes":
                correct_count += 1
            else:
                incorrect_count += 1

        if prediction == "yes":
            predicted_count += 1
        else:
            predicted_wrong_count += 1
        if answer == "yes":
            total_factual += 1
        else:
            total_not_factual += 1

        count+=1

accuracy = (correct_count + incorrect_count) / count

precision_fact = correct_count / predicted_count
precision_not_fact = incorrect_count / predicted_wrong_count
recall_fact = correct_count / total_factual
recall_not_fact = incorrect_count / total_not_factual
beta = 1
f1_fact = ((beta**2 + 1) * precision_fact * recall_fact) / \
                 ((beta**2 * precision_fact) + recall_fact)
f1_not_fact = ((beta**2 + 1) * precision_not_fact * recall_not_fact) / \
                     ((beta**2 * precision_not_fact) + recall_not_fact)

line_to_write = "{:.4}".format(accuracy) + "\r"
line_to_write += "{:.4}".format(precision_fact) + "  " + "{:.4}".format(precision_not_fact) + "\r"
line_to_write += "{:.4}".format(recall_fact) + "  " + "{:.4}".format(recall_not_fact) + "\r"
line_to_write += "{:.4}".format(f1_fact) + "  " + "{:.4}".format(f1_not_fact) + "\r"

    # Write the line to the output file
f = open("eval_NB-BOW-FV.txt", "a")
f.write(line_to_write)
f.close()
